<a href="https://colab.research.google.com/github/evansekeful/syncasanacanvas/blob/main/AsanaCanvasSync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Tasks

In [ ]:
# install required packages
%pip install asana
%pip install icalendar

In [17]:
# import libraries
import requests
import json
import pandas as pd
import asana
from icalendar import Calendar, Event
from datetime import datetime
from datetime import timezone
from pytz import all_timezones
import pytz

In [ ]:
# TODO prompt users to upload config.json

## Attention!


---


It is important that you map the location of the config.json below and that both packages above have been installed correctly before moving to the next step.

In [9]:
# read config variables into environment
config_path = "/content/config.json"

with open(config_path) as json_file:
    config = json.load(json_file)

# TODO validate config.json schema

canvas = config["canvas"]["url"]
token = config["asana"]["token"]
project = config["asana"]["project_gid"]
assignee = config["asana"]["assignee"]
workspace = config["asana"]["workspace"]

In [10]:
# set up Asana client
client = asana.Client.access_token(token)

## Run Extract + Transform

Read Canvas Calendar

In [11]:
# set up events dictionary
events = {k:[] for k in ["uid","name","start","end"]}

In [12]:
# import Canvas ics
cal = Calendar.from_ical(requests.get(canvas).text)

# set timezone to standardize timeawareness
utc = pytz.utc

# read ics into dictionary of events
for comp in cal.walk():
  if comp.get("UID") == None: continue # skip blank items
  if comp.get("SUMMARY") == None: continue # skip blank items
  if comp.get("UID").startswith("event-assignment"):
    events["uid"].append(str(comp.get("UID")))
  else: continue # skip events that are not assignments
  events["name"].append(str(comp.get("SUMMARY")))
  if hasattr(comp.get("dtstart"), "dt"):
    if comp.get("dtstart").dt.tzname() == None:
      events["start"].append(utc.localize(comp.get("dtstart").dt))
    else:
       events["start"].append(comp.get("dtstart").dt)
  if hasattr(comp.get("dtend"), "dt"):
    if comp.get("dtend").dt.tzname() == None:
      events["end"].append(utc.localize(comp.get("dtend").dt))
    else:
      events["end"].append(comp.get("dtend").dt)

In [13]:
# create extract dataframe
extract = pd.DataFrame(events)

Read Project + Clean Canvas Data

In [14]:
# set up task list
tasks = []

# set up Asana client
homework = {"project": project}
options = "notes"
result = client.tasks.get_tasks(homework,opt_fields=options)

# read homework project
for task in result:
  tasks.append(task["notes"])

In [15]:
# delete duplicates from extract table
duplicates = tuple(tasks)
extract = extract[~extract["uid"].isin(duplicates)]

In [18]:
# delete past due entries
extract = extract[extract["end"] > datetime.now(tz=timezone.utc)]

In [19]:
# delete unused columns
extract = extract[["name","uid","start","end"]]

In [23]:
# set timezone to localize due dates TODO: add to config file
extract["start"] = extract["start"].dt.tz_convert('US/Pacific')
extract["end"] = extract["end"].dt.tz_convert('US/Pacific')

Format Calendar into JSON

In [24]:
# iterate over rows to create list of dictionaries
load = []
for index, row in extract.iterrows():
  temp = {}
  temp["assignee"] = assignee
  temp["due_on"] = row["end"].strftime("%Y-%m-%d")
  temp["name"] = row["name"]
  temp["notes"] = row["uid"]
  temp["projects"] = [project]
  temp["resource_subtype"] = "default_task"
  #temp["start_on"] = row["start"].strftime("%Y-%m-%d") # paid feature
  temp["workspace"] = workspace
  load.append(temp)

In [25]:
# set up validation list
validate = []
for task in load:
  validate.append(task["notes"])

## Check Data

---

For running as an automation, be sure to wrap the load tasks with the following check.

In [26]:
len(load) > 0

True

## Run Load

Post New Assignments to Asana

In [27]:
# post new assignments to Asana
for data in load:
  result = client.tasks.create_task(data)

/usr/local/lib/python3.6/dist-packages/asana/client.py:155: UserWarning: This request is affected by the "new_user_task_lists" deprecation. Please visit this url for more info: https://forum.asana.com/t/update-on-our-planned-api-changes-to-user-task-lists-a-k-a-my-tasks/103828
Adding "new_user_task_lists" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)


Validate New Assignments Posted

In [28]:
# set up task list
tasks = []

# read homework project
result = client.tasks.get_tasks(homework,opt_fields=options)
for task in result:
  tasks.append(task["notes"])

# compare list with validation
test = set(tasks).intersection(validate)
len(validate) == len(test)

True

In [ ]:
# TODO print missing assignment ids to file